In [5]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

# Define the model architecture
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

# Compile the model (make sure to compile the model with the same configuration as during training)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Load the model from the H5 file
model = load_model('emoji.h5')

In [6]:
from keras.preprocessing.image import load_img
def ef(image):
    img = load_img(image)
    feature = np.array(img)
    feature = feature.reshape(1,48,48,3)
    return feature/255.0

In [7]:
from IPython.display import display, Javascript, Image
# from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
import PIL

import cv2
import io
import html
import time

In [8]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [9]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [10]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    // Function to remove DOM elements
    function removeDom() {
      stream.getVideoTracks()[0].stop();
      video.remove();
      div.remove();
      video = null;
      div = null;
      stream = null;
      imgElement = null;
      captureCanvas = null;
      labelElement = null;
    }

    // Function to handle key press events
    document.addEventListener('keydown', function(e) {
        if (e.key === "Escape") { // Check if the pressed key is 'Esc'
            shutdown = true; // Set shutdown flag to true
        }
    });

    // Function to handle animation frames
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    // Function to create DOM elements
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      // Create model status element
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "Status:";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      // Create video element
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      // Create image element
      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      // Create instruction element
      const instruction = document.createElement('div');
      instruction.innerHTML =
          '' +
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      // Create capture canvas element
      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }

    // Function to stream frame
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }

    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [11]:
labels = {0:'angry',1:'disgust',2:'fear',3:'happy',4:'neutral',5:'sad',6:'surprise'}
# import emoji
# labels = {0:'angry 😡',1:'disgust 🤢',2:'fear 😰',3:'happy 😊',4:'neutral 😐',5:'sad 😔',6:'surprise 😯'}
# labels = {0:"angry+str(emoji.emojize(':angry:'))",1:"disgust+str(emoji.emojize(':disgust:'))",2:"fear+str(emoji.emojize(':fear:'))",
#           3:"happy+str(emoji.emojize(':happy:'))",4:"neutral+str(emoji.emojize(':neutral:'))",5:"sad+str(emoji.emojize(':sad:'))",
#           6:"surprise+str(emoji.emojize(':surprise:'))"}

In [12]:
map = {0:'angry.png',1:'disgust.png',2:'fear.png',3:'happy.png',4:'neutral.png',
       5:'sad.png',6:'surprise.png'}

In [15]:
from google.colab.patches import cv2_imshow
# # Initialize the webcam
# webcam = cv2.VideoCapture(0)

# # Set the frame width and height
# webcam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
# webcam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Load the face cascade classifier
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

# Define a function to extract features
def extract_features(image):
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    resized_image = cv2.resize(image, (48, 48))
    feature = resized_image / 255.0
    feature = np.expand_dims(feature, axis=0)
    return feature

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # iterate over detected faces
    for (x, y, w, h) in faces:
        # extract the face region
        face_image = gray[y:y+h, x:x+w]

        # resize the face image to the required input size
        resized_image = cv2.resize(face_image, (48, 48))

        # extract features and normalize
        input_data = extract_features(resized_image)

        # make a prediction using the model
        prediction = model.predict(input_data)
        predicted_label = labels[prediction.argmax()]
        # Load the emoji image
        img_name=str(map[prediction.argmax()])
        emoji_img = cv2.imread(img_name)
        if emoji_img is not None:
            emoji_resized = cv2.resize(emoji_img, (w, h))

            # # Draw the emoji onto the frame
            # for i in range(emoji_resized.shape[0]):
            #     for j in range(emoji_resized.shape[1]):
            #         # if emoji_resized[i, j, 2] != 0:  # Check alpha channel
            #         bbox_array[y + i, x + j] = emoji_resized[i, j]
            cv2_imshow(emoji_img)
        else:
            print("Error: Unable to load the image.")

        # draw a rectangle around the face
        bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

        # display the predicted emotion label on the frame
        cv2.putText(bbox_array, predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Check for key press (ESC to exit)
    key = cv2.waitKey(1)
    if key == 27:  # ESC key
        break

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)

    # Display the frame
    # cv2.imshow("Emotion Detection", bbox_bytes)

    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

# close OpenCV windows
cv2.destroyAllWindows()

<IPython.core.display.Javascript object>

NameError: name 'eval_js' is not defined